In [1]:
import pandas as pd
import numpy as np
import pandas.io.formats.excel as excel
import os

In [2]:
#------------------------------------------------------------------------------------
original_file_path = r'D:\test_development\API\Weather_extract\Input\Input_template_time_dependent.xlsx'
file_path = r"D:\test_development\API\Weather_extract\data/"
new_file_path = r'D:\test_development\API\Weather_extract\Input\input_template_time_dependent_new.xlsx'
site_name = 'cch'
#------------------------------------------------------------------------------------

In [3]:
df = pd.read_excel(original_file_path, index_col=False, header=0, skiprows=1)

In [4]:
df_point = df.iloc[:1]
df_data = df.iloc[1:]

df_data['hour'] = df_data['*time'].astype(int)
df_data['minutes'] = np.around((df_data['*time'] - df_data['hour']) * 60).astype(int)

df_data['*To_db'] = np.nan
df_data['RH'] = np.nan

df_data = df_data.rename(columns = {'*Year':'year', '*Month':'month', '*Day_M':'day'})
df_data['date'] = pd.to_datetime(df_data[['year','month','day', 'hour', 'minutes']], format = '%Y/%m/%d %H:%M')

c:\Users\user\anaconda3\envs\sfdd\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
c:\Users\user\anaconda3\envs\sfdd\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
c:\Users\user\anaconda3\envs\sfdd\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [5]:
df_data[['*To_db', 'RH']]

,*To_db,RH
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
5,NaN,NaN
...,...,...
105404,NaN,NaN
105405,NaN,NaN
105406,NaN,NaN
105407,NaN,NaN


In [6]:
weather_df = pd.concat([pd.read_csv(file_path + data_file + '/' + site_name + '.csv') for data_file in os.listdir(file_path)], ignore_index=True)

In [7]:
weather_df

,date,humidity,temperture
0,2020/01/01 0:00,64,30.2
1,2020/01/01 0:10,63,30.3
2,2022/09/24 0:00,99,24.0
3,2022/09/24 0:10,99,24.0
4,2022/09/24 0:20,99,24.0
...,...,...,...
143,2022/09/24 23:30,79,26.1
144,2022/09/24 23:40,80,26.1
145,2022/09/24 23:50,79,25.9
146,2022/09/26 15:00,64,30.2


In [8]:
weather_df['date']=pd.to_datetime(weather_df['date'], format='%Y/%m/%d %H:%M', infer_datetime_format=True)
df_data=df_data.merge(weather_df[['date','temperture', 'humidity']],on='date',how='left')

In [9]:
df_data[['date', '*To_db', 'RH', 'temperture', 'humidity']]

,date,*To_db,RH,temperture,humidity
0,2020-01-01 00:00:00,NaN,NaN,30.2,64.0
1,2020-01-01 00:05:00,NaN,NaN,NaN,NaN
2,2020-01-01 00:10:00,NaN,NaN,30.3,63.0
3,2020-01-01 00:15:00,NaN,NaN,NaN,NaN
4,2020-01-01 00:20:00,NaN,NaN,NaN,NaN
...,...,...,...,...,...
105403,2020-12-31 23:35:00,NaN,NaN,NaN,NaN
105404,2020-12-31 23:40:00,NaN,NaN,NaN,NaN
105405,2020-12-31 23:45:00,NaN,NaN,NaN,NaN
105406,2020-12-31 23:50:00,NaN,NaN,NaN,NaN


In [10]:
df_data['*To_db'] = df_data['*To_db'].fillna(df_data.pop('temperture'))
df_data['RH'] = df_data['RH'].fillna(df_data.pop('humidity'))
df_data = df_data.drop(['hour', 'minutes', 'date'], axis = 1)
df_data = df_data.rename(columns = {'year':'*Year', 'month':'*Month', 'day':'*Day_M'})

In [11]:
df_data

,*Year,*Month,*Day_M,*Week Day,*time,*Tchws_1,Tchws_2,Tchws_3,*Tchwr_1,Tchwr_2,...,CHWP4,Pcw_P4,CWP4,P_CT4,CT4 on/ff,#Tcwi CT4,#Tcwo CT4,#Mcw CT4,Mca CT4,CT4 Valve
0,2020.0,1.0,1.0,3.0,0.00,7.285,NaN,NaN,10.345,NaN,...,OFF,0,OFF,0,OFF,NaN,NaN,NaN,0,NaN
1,2020.0,1.0,1.0,3.0,0.08,7.245,NaN,NaN,10.395,NaN,...,OFF,0,OFF,0,OFF,NaN,NaN,NaN,0,NaN
2,2020.0,1.0,1.0,3.0,0.17,7.3,NaN,NaN,10.34,NaN,...,OFF,0,OFF,0,OFF,NaN,NaN,NaN,0,NaN
3,2020.0,1.0,1.0,3.0,0.25,7.3,NaN,NaN,10.43,NaN,...,OFF,0,OFF,0,OFF,NaN,NaN,NaN,0,NaN
4,2020.0,1.0,1.0,3.0,0.33,7.225,NaN,NaN,10.205,NaN,...,OFF,0,OFF,0,OFF,NaN,NaN,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105403,2020.0,12.0,31.0,4.0,23.58,7.385,NaN,NaN,10.145,NaN,...,OFF,0,OFF,0,OFF,NaN,NaN,NaN,0,NaN
105404,2020.0,12.0,31.0,4.0,23.67,7.325,NaN,NaN,9.94,NaN,...,OFF,0,OFF,0,OFF,NaN,NaN,NaN,0,NaN
105405,2020.0,12.0,31.0,4.0,23.75,7.25,NaN,NaN,9.915,NaN,...,OFF,0,OFF,0,OFF,NaN,NaN,NaN,0,NaN
105406,2020.0,12.0,31.0,4.0,23.83,7.27,NaN,NaN,9.945,NaN,...,OFF,0,OFF,0,OFF,NaN,NaN,NaN,0,NaN


In [17]:
writer = pd.ExcelWriter(r'D:\test_development\API\Weather_extract\Input\input_template_time_dependent_new.xlsx', engine = 'xlsxwriter')

In [18]:
excel.ExcelFormatter.header_style = None

In [19]:
df_point.to_excel(writer, sheet_name='Sheet1', startrow = 0, index=False, header = False)

In [20]:
df_data.to_excel(writer, sheet_name='Sheet1', startrow = 1, index=False, header = True)

In [21]:
writer.save()